EXPORTING DATASET TO AWS DYNAMODB

In [ ]:
import pandas as pd
import boto3

# DynamoDB table

# Dataset definition
original_dataset = pd.read_csv('data/hotel_original.csv')

# VAR AWS Region
region_name = 'us-east-1'

# VAR AWS DynamoDB Table
dynamodb_table_name = 'hotel_reservation_dataset_original'  

# Send to DynamoDb
dynamodb = boto3.resource('dynamodb', region_name)

items = []
for index, row in original_dataset.iterrows():
    item = {
        'Booking_ID': {'S': (row['Booking_ID'])},
        'no_of_adults': {'N': str(row['no_of_adults'])},
        'no_of_children': {'N': str(row['no_of_children'])},
        'no_of_weekend_nights': {'N': str(row['no_of_weekend_nights'])},
        'no_of_week_nights': {'N': str(row['no_of_week_nights'])},
        'type_of_meal_plan': {'S': row['type_of_meal_plan']},
        'required_car_parking_space': {'N': str(row['required_car_parking_space'])},
        'room_type_reserved': {'S': row['room_type_reserved']},
        'lead_time': {'N': str(row['lead_time'])},
        'arrival_year': {'N': str(row['arrival_year'])},
        'arrival_month': {'N': str(row['arrival_month'])},
        'arrival_date': {'N': str(row['arrival_date'])},
        'market_segment_type': {'S': row['market_segment_type']},
        'repeated_guest': {'N': str(row['repeated_guest'])},
        'no_of_previous_cancellations': {'N': str(row['no_of_previous_cancellations'])},
        'no_of_previous_bookings_not_canceled': {'N': str(row['no_of_previous_bookings_not_canceled'])},
        'avg_price_per_room': {'N': str(row['avg_price_per_room'])},
        'no_of_special_requests': {'N': str(row['no_of_special_requests'])},
        'booking_status': {'S': row['booking_status']}
    }
    items.append(item)

dynamodb = boto3.client('dynamodb', region_name)
for item in items:
    response = dynamodb.put_item(
        TableName=dynamodb_table_name,
        Item=item
    )

IMPORTING DATASET FROM DYNAMODB

In [ ]:
# VAR AWS Region
region_name = 'us-east-1'

# VAR AWS DynamoDB Table Original Dataset
dynamodb_table_name = 'hotel_reservation_dataset_original' 

# Table
dynamodb = boto3.resource('dynamodb', region_name)
table = dynamodb.Table(dynamodb_table_name)

all_items = []
start_key = None

while True:
    #  Pagination
    if start_key:
        response = table.scan(ExclusiveStartKey=start_key)
    else:
        response = table.scan()

    # Add imported items to list
    items_import = response['Items']
    all_items.extend(items_import)

    #  Check if pagination over
    if 'LastEvaluatedKey' in response:
        start_key = response['LastEvaluatedKey']
    else:
        break
    
df = pd.DataFrame(all_items)
df.shape

CORRRECTING DATA TYPE

In [ ]:
# Only in case import from DynamoDB comes with JSON specifying all types as string

string_to_int = ["no_of_previous_bookings_not_canceled", "no_of_children", "repeated_guest", "no_of_special_requests", "no_of_adults", "arrival_year", "no_of_week_nights", "required_car_parking_space", "arrival_date", "no_of_weekend_nights", "arrival_month", "lead_time", "no_of_previous_cancellations"]

for column in string_to_int:
    df[column] = df[column].astype(int)
    
df["avg_price_per_room"] = df["avg_price_per_room"].astype(float)

print(df.dtypes)